## Image Augmentations

In [6]:
import os
import numpy as np
from PIL import Image, ImageFilter
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# === Suppress sklearn warnings ===
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# === Augmentation Functions ===
def apply_noise(images, mean=0, std=10, seed=3888):
    if std > 0:
        np.random.seed(seed)
        noise = np.random.normal(mean, std, images.shape)
        noisy_images = images + noise
        noisy_images = np.clip(noisy_images, 0, 255).astype(np.uint8)
        return noisy_images
    else:
        return images

def apply_blur(images, size):
    size = int(size)
    blurred = []
    if size > 0:
        for img in images:
            pil_img = Image.fromarray(img)
            blur = pil_img.filter(ImageFilter.GaussianBlur(radius=size))
            blurred.append(np.array(blur))
        return np.array(blurred).astype(np.uint8)
    else:
        return images

# === Settings ===
source_dir = "images"
output_dir = "images_augmented"
blur_levels = [0, 1, 3, 5, 7, 9, 19]
noise_levels = [0, 1, 3, 5, 10, 20, 30]

image_files = [f for f in os.listdir(source_dir) if f.endswith(".png")]

# === Main Augmentation Loop ===
for img_name in image_files:
    img_path = os.path.join(source_dir, img_name)
    img = Image.open(img_path).convert("RGB")
    img_array = np.array(img).astype(np.uint8)
    img_batch = img_array[np.newaxis, ...]

    for blur in blur_levels:
        for noise in noise_levels:
            # === 1. Blur Only ===
            if noise == 0:
                blurred = apply_blur(img_batch, blur)[0]
                folder = f"{output_dir}/blur_only/blur_{blur}"
                os.makedirs(folder, exist_ok=True)
                Image.fromarray(blurred).save(os.path.join(folder, img_name))

            # === 2. Noise Only ===
            if blur == 0:
                noised = apply_noise(img_batch, std=noise)[0]
                folder = f"{output_dir}/noise_only/noise_{noise}"
                os.makedirs(folder, exist_ok=True)
                Image.fromarray(noised).save(os.path.join(folder, img_name))

            # === 3. Blur + Noise Combo ===
            blurred = apply_blur(img_batch, blur)
            blurred_noised = apply_noise(blurred, std=noise)[0]

            combo_folder = f"{output_dir}/combo/{blur}_blur_{noise}_noise"
            os.makedirs(combo_folder, exist_ok=True)
            Image.fromarray(blurred_noised).save(os.path.join(combo_folder, img_name))